# **AMAZON REVIEW SCRAPER 2024**


## Beschreibung:

Dieses Skript umfasst einen Amazon-Review-Webscraper, welcher die Limitationen von Amazon (nur 100 sichtbare Rezensionen pro Produkt) durch Anwendung verschiedener Filter umgeht. So ist es Möglich bis zu 1500 Produktrezensionen pro Produktvariation (z.B. Größe) zu scrapen. Die Informationen: 
1. Kundenname
2. Reviewtitel
3. Herkunftsland
4. Datum
5. Produktvariation
6. Review-Text
7. Sternebewertung
8. Anzahl der Likes

werden bei der Ausführung einzeln extrahiert und in eine CSV-Datei eingetragen, um nach dem Scraping Prozess weiter verwendet werden zu können.

Um das Skript nutzen zu können sind einige Voraussetzungen notwendig. Im Folgenden wird eine vollständige Anleitung gegeben.



## Voraussetzungen


In [1]:
# ZU BEGINN: Lade notwendige Packages
# pip install pandas
# pip install beautifulsoup4
# pip install selenium

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
# Selenium benutzt einen Webdriver!
# Dieser muss vorher heruntergeladen werden!

# ANLEITUNG:

# 1. Downloade kompatiblen Driver

#Chrome:  https://chromedriver.chromium.org
#Firefox: https://github.com/mozilla/geckodriver/releases
#Safari:  https://webkit.org/blog/6900/webdriver-support-in-safari-10/

# 2. Extrahiere den Webdriver aus der Zip-Datei und speichere den Webdriver unter:
# WINDOWS:  C:\Windows\System32 
# MAC: usr/local/bin 

# FERTIG!

In [3]:
# Um auf Mehrere Rezensionsseiten eines Produktes zugreifen zu können, wird ein Amazon-Account benötigt:

# Wenn nicht der Privataccount genutzt werden soll:

#1. Anonyme Email-Adresse erstellen unter: https://tuta.com/de/

#2. Amazon Account mit anonymer Email-Adresse erstellen unter: https://www.amazon.de/ap/register?openid.mode=checkid_setup&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&openid.return_to=https%3A%2F%2Fwww.amazon.de%2Fref%3Drhf_sign_in&openid.assoc_handle=deflex


# Benutzerdaten eingeben
email = "MaxMustermann@tutamail.com"
password = "passwort"

# URL des Produktes Eingeben (von dem die Rezensionen verlangt werden)
url = 'https://www.amazon.de/Bonsai-Starter-Kit-Bonsai-Anzuchtset-wundersch%C3%B6nen/dp/B08BTZK7CT/ref=sr_1_2_sspa?__mk_de_DE=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2N60HUUWW0MQM&dib=eyJ2IjoiMSJ9.N3FS1MXb-D1UIuWY5nEt9FfyMB-9NMrcIHlkdQg9L-Wqk3029rUW-1oDPXeLtfQJPXSaRSMeBfcZ73nNs5ONVhwIqhc08rniHoxI_uRQq5uscIsQMPfV25R54overiMNK4wmo3Mw7a2MuJcx-sSxCLjPQNVLbUP8B4ipgpLHsVima73jlSX6htxuWuKUw1I2NH5_EbLYknX3-0L6x9dcsbPEMcJobKS-tQB7GldTqmANKPud5dYCGj8vhCsK_2tm5cNEptv0GVjicmwa6tPpUaH1vltgIRWg0zkp5mc5SIo.OisVgWYO63WeWeq8OCGQ0Sh062UuTBxaQuMU8Hma3Ko&dib_tag=se&keywords=bonsai&qid=1712178520&sprefix=bonsai%2Caps%2C93&sr=8-2-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&th=1'

## Funktionen und Definition der Filterkriterien

In [4]:
# SCRAPING FUNKTION (immer für eine aufgerufene Seite)

def ama_scrape(soup):
    reviews = []
    # Finde alle Reviews auf der Seite
    for review in soup.findAll('div', class_='review'):
        # Extrahiere User-Name
        name_user = review.find('span', class_='a-profile-name').text.strip()
        # Extrahiere Titel des Reviews
        if review.find('span',class_='').text.strip() == 'Melden':  # Falls kein Titel vorhanden ist, wird der nächste <span> genommen -> verhindert
            titel = ''
        else:
            titel = review.find('span',class_='').text.strip()
        # Extrahiere Datum und Land
        datum_land = review.find('span', class_= 'a-size-base a-color-secondary review-date').text.strip()
        if len(datum_land.split(' vom ',1)) == 2:
            land, datum = datum_land.split(' vom ',1)
            land = land.split()[-1]
        else:
            land = ''
            datum = ''
        # Extrahiere die Produktausprägung
        variation = review.find('a', attrs={'data-hook':'format-strip'})
        if variation and variation.text:
            variation = review.find('a', attrs={'data-hook':'format-strip'}).text.strip()
        else:
            variation = ''
        # Extrahiere den Review Text
        text = review.find('span', class_='review-text').text.strip()
        # Extrahiere die Sterne-Bewertung
        stars = review.find('i', class_='review-rating').text.strip()[0]
        # Extrahiere die 'Hilfreich'-Angaben
        hilfreich = review.find('span', class_='a-size-base a-color-tertiary cr-vote-text')
        if hilfreich and hilfreich.text:
            hilfreich = review.find('span', class_='a-size-base a-color-tertiary cr-vote-text').text.strip().split()[0]
            if hilfreich == 'Eine':
                hilfreich = 1
        else:
            hilfreich = '0'

        # Review-Daten in Liste speichern
        reviews.append({
            'Name' : name_user,
            'Titel': titel,
            'Land': land,
            'Datum': datum,
            'Variation': variation,
            'Text': text,
            'Sterne': stars,
            'Hilfreich': hilfreich
        })

    # DataFrame aus der Liste erstellen
    df = pd.DataFrame(reviews)

    return df


In [5]:
# Dataframe zum sammeln der Reviews
complete_reviews = pd.DataFrame()

# FILTERREGISTER (Filtert später die Rezensionen -> Für jedes Produkt gleich!)
# Kann bei Aktualisierungen Seiten Amazons angepasst werden.

# neueste,verifiziert,alle sterne,alle formate
filter_1 ='ref=cm_cr_arp_d_viewopt_rvwer?ie=UTF8&reviewerType=avp_only_reviews&sortBy=recent&pageNumber=1'
# neueste,verifiziert,kritische,alle formate
filter_2 = 'ref=cm_cr_arp_d_viewopt_sr?ie=UTF8&reviewerType=avp_only_reviews&sortBy=recent&pageNumber=1&filterByStar=critical'
# neueste,verifiziert,positive,alle formate
filter_3 = 'ref=cm_cr_arp_d_viewopt_sr?ie=UTF8&reviewerType=avp_only_reviews&sortBy=recent&pageNumber=1&filterByStar=positive'
# neueste,verifiziert,1 stern,alle formate
filter_4 = 'ref=cm_cr_arp_d_viewopt_sr?ie=UTF8&reviewerType=avp_only_reviews&sortBy=recent&pageNumber=1&filterByStar=one_star'
# neueste,verifiziert,2 sterne,alle formate
filter_5 = 'ref=cm_cr_arp_d_viewopt_sr?ie=UTF8&reviewerType=avp_only_reviews&sortBy=recent&pageNumber=1&filterByStar=two_star'
# neueste,verifiziert,3 sterne,alle formate
filter_6 = 'ref=cm_cr_arp_d_viewopt_sr?ie=UTF8&reviewerType=avp_only_reviews&sortBy=recent&pageNumber=1&filterByStar=three_star'
# neueste,verifiziert,4 sterne,alle formate
filter_7 = 'ref=cm_cr_arp_d_viewopt_sr?ie=UTF8&reviewerType=avp_only_reviews&sortBy=recent&pageNumber=1&filterByStar=four_star'
# neueste,verifiziert,5 sterne,alle formate
filter_8 = 'ref=cm_cr_arp_d_viewopt_sr?ie=UTF8&reviewerType=avp_only_reviews&sortBy=recent&pageNumber=1&filterByStar=five_star'
# spitzen,verifiziert,kritisch,alle formate
filter_9 = 'ref=cm_cr_arp_d_viewopt_sr?ie=UTF8&reviewerType=avp_only_reviews&sortBy=helpful&pageNumber=1&filterByStar=critical'
# spitzen,verifiziert,positiv,alle formate
filter_10 = 'ref=cm_cr_arp_d_viewopt_sr?ie=UTF8&reviewerType=avp_only_reviews&sortBy=helpful&pageNumber=1&filterByStar=positive'
# spitzen,verifiziert,1 Stern,alle formate
filter_11 = 'ref=cm_cr_arp_d_viewopt_sr?ie=UTF8&reviewerType=avp_only_reviews&sortBy=helpful&pageNumber=1&filterByStar=one_star'
# spitzen,verifiziert,2 Sterne,alle formate
filter_12 = 'ref=cm_cr_arp_d_viewopt_sr?ie=UTF8&reviewerType=avp_only_reviews&sortBy=helpful&pageNumber=1&filterByStar=two_star'
# spitzen,verifiziert,3 Sterne,alle formate
filter_13 = 'ref=cm_cr_arp_d_viewopt_sr?ie=UTF8&reviewerType=avp_only_reviews&sortBy=helpful&pageNumber=1&filterByStar=three_star'
# spitzen,verifiziert,4 Sterne,alle formate
filter_14 = 'ref=cm_cr_arp_d_viewopt_sr?ie=UTF8&reviewerType=avp_only_reviews&sortBy=helpful&pageNumber=1&filterByStar=four_star'
# spitzen,verifiziert,5 Sterne,alle formate
filter_15 = 'ref=cm_cr_arp_d_viewopt_sr?ie=UTF8&reviewerType=avp_only_reviews&sortBy=helpful&pageNumber=1&filterByStar=five_star'

# LISTE ALLER FILTER
all_filters = [filter_1,filter_2,filter_3,filter_4,filter_5,filter_6,filter_7,filter_8,filter_9,filter_10,filter_11,filter_12,filter_13,filter_14,filter_15,]

In [6]:
# FUNKTION ZUR ANWENDUNG ALLER FILTER
def ama_scrape_with_filters(reviews_url,filters):

    # URL aufspalten um Filter anzuwenden
    base_url= reviews_url.split('/ref=')[0]

    # Dataframe um Reviews innerhalb der Funktion zu speichern
    all_reviews = pd.DataFrame()

    for filter in all_filters:

        #Link nach Filter anpassen
        current_url = base_url + '/' + filter

        # Scaping aller Seiten für eine Filteroption
        for page in range(1,11): # Amazon stellt immer nur 10 Seiten zur Einsicht
            if page == 1:
                driver.get(current_url)
                html = driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                current_reviews = ama_scrape(soup)
            else:
                # Abfage ob es eine weitere Seite gibt
                if (soup.find('li', class_='a-last') == None) or (soup.find('li', class_='a-last').find('a') == None):
                    break
                # Wenn es eine weitere Seite gibt
                new_current_url = base_url + soup.find('li', class_='a-last').find('a')['href']
                driver.get(new_current_url)
                html = driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                current_reviews = ama_scrape(soup)

            # Reviews in Dataframe schreiben
            all_reviews = pd.concat([all_reviews, current_reviews], ignore_index=True)

    return all_reviews

## Scraping Prozess Step by Step


In [7]:
# EINLOGGEN UND AUF REZENSIONSSEITE GEHEN

# Initialisierung des Webdrivers
driver = webdriver.Firefox() # Alternativ webdriver.Chrome(), wenn WebDriver nicht unter C:\Windows\System32 gespeichert ist Pfad angeben! ->webdriver.Chrome('Pfad')
driver.get(url)

# Funktion um zu warten, bis die Seite geladen ist
wait = WebDriverWait(driver, 10)

# Ausgangspunkt: PRODUKTSEITE

# Auf Sign-in Seite gehen
sign_in_field = wait.until(EC.visibility_of_element_located((By.ID,"nav-link-accountList")))
sign_in_field.click()

# E-Mail eingeben und auf "Weiter" klicken
email_field = wait.until(EC.visibility_of_element_located((By.ID, "ap_email")))
email_field.send_keys(email)

continue_button = wait.until(EC.element_to_be_clickable((By.ID, "continue")))
continue_button.click()

# Passwort eingeben und auf "Anmelden" klicken (Dauert idR. einige Sekunden)
password_field = wait.until(EC.visibility_of_element_located((By.ID, "ap_password")))
password_field.send_keys(password)

sign_in_button = wait.until(EC.element_to_be_clickable((By.ID, "signInSubmit")))
sign_in_button.click()

In [12]:
# Übergang zu Rezensionsseite

# WICHTIG:
# Wenn eine bestimmte Variation des Produktes (Größe etc.) betrachtet werden soll -> im Webdriver anklicken
# Wenn mehrere Variationen betrachtet werden sollen, müssen folgende Schritte Mehrfach durchgeführt werden -> Später im Skript

# Link zur Rezensionsseite finden
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
reviews_url = 'https://www.amazon.de' + soup.findAll('a', class_='a-link-emphasis a-text-bold')[0]['href']
driver.get(reviews_url)


In [13]:
# Alle verfügbaren Reviews aller Seiten und aller relevanten Filter scrapen
current_variation_reviews = ama_scrape_with_filters(reviews_url, all_filters)

# Falls weitere Produktvariationen abgefragt werden sollen -> In Sammeldataframe speichern
complete_reviews = pd.concat([complete_reviews, current_variation_reviews], ignore_index=True)

---------------------- Dieser Schritt ist beliebig oft wiederholbar------------------------


In [ ]:
# Falls weitere Produktvariationen gescraped werden sollen:

# Zur Produktseite zurückkehren
driver.get(url)

# !!!Nun bitte gewünschte Variation manuell anklicken!!!


In [ ]:
# Link zur Rezensionsseite finden
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
reviews_url = 'https://www.amazon.de' + soup.findAll('a', class_='a-link-emphasis a-text-bold')[0]['href']
driver.get(reviews_url)

# Alle verfügbaren Reviews aller Seiten und aller relevanten Filter scrapen
current_variation_reviews = ama_scrape_with_filters(reviews_url, all_filters)

# Falls weitere Produktvariationen abgefragt werden sollen -> In Sammeldataframe speichern
complete_reviews = pd.concat([complete_reviews, current_variation_reviews], ignore_index=True)

---------------------- Ende der Iteration------------------------

In [14]:
# Alle gewünschten Reviews beisammen? Wenn ja:

# Abschließend Duplikate entfernen
reviews_cleaned = complete_reviews.drop_duplicates()

#Reviews in CSV-Datei speichern
reviews_cleaned.to_csv('amazon_reviews.csv', index=False, encoding='utf-8-sig')

# Webdriver schließen
driver.quit()